In [ ]:
def negate(clause):
    return tuple(el.strip("~") if "~" in el else f"~{el}" for el in clause)

In [ ]:
negate(("~a", "~b", "~c", "~d", "e","f"))

In [ ]:
def is_tautology(clause):
    for literal in clause:
        if literal.startswith("~"):
            if literal[1:] in clause:
                return True
        else:
            if f"~{literal}" in clause:
                return True
    return False

In [ ]:
def parse_clause_file(path_to_clause_file):
    clauses = {}
    file = open(path_to_clause_file, 'r')
    line_number = 0
    while True:
        line = file.readline()
        if not line:
            file.close()
            break
        if line.startswith("#"):
            continue
        line_number += 1
        line_elements = line.strip().split(" ")
        line_tuple = tuple(sorted(element.lower() for element in line_elements if element.lower() != "v"))
        if is_tautology(line_tuple):
            line_number -= 1
            continue
        clauses[line_tuple] = line_number
        target_clause = line_tuple
    del clauses[target_clause]
    for clause in negate(target_clause):
        clauses[clause] = line_number
        line_number += 1
    sorted_clauses = {key: clauses[key] for key in sorted(clauses)}
    return sorted_clauses, target_clause, line_number

In [ ]:
hola = parse_clause_file(r'C:\FER\6TH SEMESTER\INTRO_TO_AI\autograder\data\lab2\files\cooking_chicken_broccoli_alfredo_big.txt')

In [ ]:
negate(("~a", "~b", "~c", "~d", "e","f"))

In [ ]:
hola[0]

In [ ]:
def remove_redundant_clauses(clauses):
    non_redundant_clauses = set()
    for clause in clauses:
        clause_set = set(clause)
        if not any(clause_set != set(other_clause) and clause_set.issubset(set(other_clause)) for other_clause in clauses):
            non_redundant_clauses.add(clause)
    return non_redundant_clauses


In [ ]:
def plResolve(clause_1, clause_2,clauses):
    clause1_set = set(clause_1)
    clause2_set = set(clause_2)
    resolvents = set()

    for element in clause1_set:
        complementary_element = element[1:] if element.startswith("~") else f"~{element}"
        if complementary_element in clause2_set:
            new_clause = clause1_set.union(clause2_set) - {element, complementary_element}
            if not new_clause: 
               return ("NIL",)
            if is_tautology(new_clause):
               continue
            if tuple(sorted(new_clause)) not in clauses.keys():
                resolvents.add(tuple(sorted(new_clause)))
    return sorted(resolvents, key=len)
 

In [ ]:
def resolution(clauses, target_clause, line_num):
    sos = set([negate(target_clause)])   
    
    while True:
        new = set()
        for id,clause_1 in enumerate(sos):
            for clause_2 in clauses.keys():
               # print(f"SOS: {sos},clauses : {clauses.keys()}")
               resolvents = plResolve(clause_1, clause_2,clauses)
               if "NIL" in resolvents: 
                  return True, clause_1, clause_2, None, None
               new.update(resolvents)
            for id2,previous_clause in enumerate(sos):
                if id2>=id:
                   break
                if previous_clause == clause_1:
                    break  
                resolvents = plResolve(clause_1, previous_clause,clauses)
                if "NIL" in resolvents: 
                    return True, clause_1, previous_clause, None, None
                new.update(resolvents)
        new = remove_redundant_clauses(new)
        if new.issubset(clauses.keys()) or len(new)==0:
            return False, None, None, None, None
        l = len(sos)
        sos.update(new)
        sos = remove_redundant_clauses(sos)
        l2 = len(sos)
        if l==l2:
            return False, None, None, None, None

In [ ]:
resolution(hola[0],hola[1],hola[2])

In [ ]:
if 1==1:
      clauses,target_clause,line_num = parse_clause_file(r'C:\FER\6TH SEMESTER\INTRO_TO_AI\autograder\data\lab2\files\resolution_heldout_goal_isnegated.txt')
      found,cl1,cl2,ln1,ln2= resolution(clauses,target_clause,line_num)
      target_clause_f = ""
      i = 0 
      for el in target_clause:
         if i == 0:
            target_clause_f = target_clause_f + str(el)
            i+=1
         else:
            target_clause_f+=" v " + str(el)
      if found:
         print(f"[CONCLUSION]: {target_clause_f} is true")
      else:
         print(f"[CONCLUSION]: {target_clause_f} is unknown")

In [50]:
import sys
import argparse
import itertools

def parse_command_line():
   parser = argparse.ArgumentParser()
   parser.add_argument('task',type=str,choices=['resolution','cooking'],help="Type of task to run")
   parser.add_argument('path_to_clause_file',type=str,help="Path to the file with the list of clauses")
   parser.add_argument('path_to_user_commands_file',type=str,nargs='?',help="Path to the file with a list of user commands")
   args = parser.parse_args()
   return args

def parse_user_commands_file(path_to_user_commands_file):
   pass

def negate(clause):
    return tuple(el.strip("~") if "~" in el else f"~{el}" for el in clause)
 
def is_tautology(clause):
    for literal in clause:
        if literal.startswith("~"):
            if literal[1:] in clause:
                return True
        else:
            if f"~{literal}" in clause:
                return True
    return False

def remove_redundant_clauses(clauses):
    non_redundant_clauses = set()
    for clause in clauses:
        clause_set = set(clause)
        if not any(clause_set != set(other_clause) and clause_set.issubset(set(other_clause)) for other_clause in clauses):
            non_redundant_clauses.add(clause)
    return non_redundant_clauses


def parse_clause_file(path_to_clause_file):
    clauses = {}
    file = open(path_to_clause_file, 'r')
    line_number = 0
    while True:
        line = file.readline()
        if not line:
            file.close()
            break
        if line.startswith("#"):
            continue
        line_number += 1
        line_elements = line.strip().split(" ")
        line_tuple = tuple(sorted(element.lower() for element in line_elements if element.lower() != "v"))
        if is_tautology(line_tuple):
            line_number -= 1
            continue
        clauses[line_tuple] = line_number
        target_clause = line_tuple
    orig = target_clause
    del clauses[target_clause]
    target_clause_negated = {}
    for clause in negate(target_clause):
        negated_clause_tuple = (clause,)
        target_clause_negated[negated_clause_tuple] = line_number
        line_number += 1
    sorted_clauses = {key: clauses[key] for key in sorted(clauses)}
    return sorted_clauses, target_clause_negated, line_number,orig

def plResolve(clause_1, clause_2,clauses):
    clause1_set = set(clause_1)
    clause2_set = set(clause_2)
    resolvents = set()

    for element in clause1_set:
        complementary_element = element[1:] if element.startswith("~") else f"~{element}"
        if complementary_element in clause2_set:
            new_clause = clause1_set.union(clause2_set) - {element, complementary_element}
            if not new_clause: 
               return ("NIL",)
            if is_tautology(new_clause):
               continue
            if tuple(sorted(new_clause)) not in clauses.keys():
                resolvents.add(tuple(sorted(new_clause)))
    # print(resolvents)
    return sorted(resolvents, key=len)

def resolution(clauses, target_clause_negated, line_num):
    sos = set(target_clause_negated.keys())
    new = set()

    while True:
        for clause1 in sos:
            for clause2 in itertools.chain(clauses.keys(), sos):
                if clause1 != clause2:
                    resolvents = plResolve(clause1, clause2, clauses)
                    if "NIL" in resolvents:
                        return True, clause1, clause2, None, None
                    new.update(resolvents)

        new = remove_redundant_clauses(new - sos)
        if not new:
            return False, None, None, None, None

        sos.update(new)

        

def cooking(path_to_clause_file,path_to_user_commands_file):
   pass

def main():
   args = parse_command_line()
   if args.task=='resolution':
      clauses,target_clause_negated,line_num,target_clause = parse_clause_file(args.path_to_clause_file)
      found,cl1,cl2,ln1,ln2= resolution(clauses,target_clause_negated,line_num)
      target_clause_f = ""
      i = 0 
      for el in target_clause:
         if i == 0:
            target_clause_f = target_clause_f + str(el)
            i+=1
         else:
            target_clause_f+=" v " + str(el)
      if found:
         print(f"[CONCLUSION]: {target_clause_f} is true")
      else:
         print(f"[CONCLUSION]: {target_clause_f} is unknown")
   elif args.task=='cooking':
      cooking(args.path_to_clause_file,args.path_to_user_commands_file)


In [57]:
if 1==1:
      clauses,target_clause,line_num,_= parse_clause_file(r'C:\FER\6TH SEMESTER\INTRO_TO_AI\autograder\data\lab2\files\new_example_1.txt')
      found,cl1,cl2,ln1,ln2= resolution(clauses,target_clause,line_num)
      target_clause_f = ""
      i = 0 
      for el in target_clause:
         if i == 0:
            target_clause_f = target_clause_f + str(el)
            i+=1
         else:
            target_clause_f+=" v " + str(el)
      if found:
         print(f"[CONCLUSION]: {target_clause_f} is true")
      else:
         print(f"[CONCLUSION]: {target_clause_f} is unknown")

[CONCLUSION]: ('~c',) is unknown


In [42]:
def parse_clause_file(path_to_clause_file):
    clauses = {}
    file = open(path_to_clause_file, 'r')
    line_number = 0
    while True:
        line = file.readline()
        if not line:
            file.close()
            break
        if line.startswith("#"):
            continue
        line_number += 1
        line_elements = line.strip().split(" ")
        line_tuple = tuple(sorted(element.lower() for element in line_elements if element.lower() != "v"))
        if is_tautology(line_tuple):
            line_number -= 1
            continue
        clauses[line_tuple] = line_number
        target_clause = line_tuple
    orig = tuple(target_clause)
    del clauses[target_clause]
    target_clause_negated = {}
    for clause in negate(target_clause):
        negated_clause_tuple = (clause,)
        target_clause_negated[negated_clause_tuple] = line_number
        line_number += 1
    sorted_clauses = {key: clauses[key] for key in sorted(clauses)}
    return sorted_clauses, target_clause_negated, line_number,orig

In [43]:
rez = parse_clause_file(r'C:\FER\6TH SEMESTER\INTRO_TO_AI\autograder\data\lab2\files\cooking_chicken_broccoli_alfredo_big.txt')
print(rez[0])
print("--------------------------------------------------------")
print(rez[1])

{('alfredo_sauce', '~butter', '~cream', '~flour', '~milk', '~parmesan'): 12, ('broccoli',): 2, ('butter',): 5, ('chicken',): 4, ('cream',): 8, ('diced_chicken', '~chicken', '~olive_oil'): 11, ('flour',): 6, ('milk',): 7, ('olive_oil',): 3, ('parmesan',): 9, ('parsley',): 10, ('pasta',): 1}
--------------------------------------------------------
{('~chicken_broccoli_alfredo',): 13, ('alfredo_sauce',): 14, ('broccoli',): 15, ('diced_chicken',): 16, ('pasta',): 17}


In [39]:
for le in rez[0]:
   print(le)

('alfredo_sauce', '~butter', '~cream', '~flour', '~milk', '~parmesan')
('broccoli',)
('butter',)
('chicken',)
('cream',)
('diced_chicken', '~chicken', '~olive_oil')
('flour',)
('milk',)
('olive_oil',)
('parmesan',)
('parsley',)
('pasta',)


In [40]:
for le in rez[1]:
   print(le)

('~chicken_broccoli_alfredo',)
('alfredo_sauce',)
('broccoli',)
('diced_chicken',)
('pasta',)


In [44]:
for el in rez[3]:
   print(el)

chicken_broccoli_alfredo
~alfredo_sauce
~broccoli
~diced_chicken
~pasta


In [46]:
def main():
   if 1==1:
      line_num=2
      found,cl1,cl2,ln1,ln2= resolution(rez[0],rez[1],line_num)
      target_clause_f = ""
      i = 0 
      for el in rez[3]:
         if i == 0:
            target_clause_f = target_clause_f + str(el)
            i+=1
         else:
            target_clause_f+=" v " + str(el)
      if found:
         print(f"[CONCLUSION]: {target_clause_f} is true")
      else:
         print(f"[CONCLUSION]: {target_clause_f} is unknown")
main()

[CONCLUSION]: chicken_broccoli_alfredo v ~alfredo_sauce v ~broccoli v ~diced_chicken v ~pasta is unknown
